# Full Fine-Tuning with Unsloth: Qwen2.5-7B on AG News

This notebook demonstrates **Full Fine-Tuning** using **Unsloth's FastLanguageModel** for optimized performance on DGX Spark.

## Overview

| Aspect | Details |
|--------|---------|
| **Model** | unsloth/Qwen2.5-7B-Instruct |
| **Method** | Full Fine-Tuning (100% parameters) |
| **Framework** | Unsloth + TRL |
| **Dataset** | AG News (120K train, 7.6K test) |
| **Task** | 4-class text classification |
| **Expected Time** | ~4-5 hours (with Unsloth optimizations) |
| **Memory** | ~60-70 GB |
| **Output** | Full model (~14 GB) |

## Base Model Performance (Target to Beat)

| Metric | Base Model | Target |
|--------|------------|--------|
| **Accuracy** | 78.76% | >88% |
| **F1 (macro)** | 77.97% | >85% |
| **Sci/Tech F1** | 62.06% | >80% |
| **Business Precision** | 63.66% | >80% |

## Full Fine-Tuning vs LoRA

| Aspect | Full Fine-Tuning | LoRA |
|--------|------------------|------|
| Parameters trained | 7.6B (100%) | ~70M (1%) |
| Memory usage | ~60-70 GB | ~25 GB |
| Output size | ~14 GB | ~200 MB |
| Quality | Maximum | Slightly lower |

## Prerequisites

```bash
./start_docker.sh start finetune
# Then open http://localhost:8888
```

**Note**: Full fine-tuning requires significant memory. DGX Spark's 128 GB unified memory is sufficient.

## 1. Environment Setup

In [ ]:
import torch
import os

print("=" * 60)
print("Environment Verification - Full Fine-Tuning")
print("=" * 60)

print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Compute Capability: {torch.cuda.get_device_capability(0)}")
    try:
        total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU Memory: {total_mem:.1f} GB")
    except:
        print("GPU Memory: Unified memory system (DGX Spark)")
else:
    raise RuntimeError("CUDA not available!")

print(f"\nWorking directory: {os.getcwd()}")
print(f"Dataset available: {os.path.exists('/fine-tuning-dense/datasets/train.jsonl')}")

## 2. Configuration

In [ ]:
# =============================================================================
# Model Configuration
# =============================================================================
MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct"
MAX_SEQ_LENGTH = 512

# =============================================================================
# Training Configuration
# =============================================================================
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-5  # Lower than LoRA for full fine-tuning
NUM_EPOCHS = 1
WARMUP_RATIO = 0.03
WEIGHT_DECAY = 0.01

# =============================================================================
# Output Configuration
# =============================================================================
OUTPUT_DIR = "./checkpoints/qwen7b-ag-news-full-unsloth"
LOGGING_STEPS = 50
SAVE_STEPS = 500

TRAIN_DATA_PATH = "/fine-tuning-dense/datasets/train.jsonl"

print("Full Fine-Tuning Configuration loaded!")
print(f"  Model: {MODEL_NAME}")
print(f"  Method: Full Fine-Tuning (100% parameters)")
print(f"  Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Output: {OUTPUT_DIR}")

## 3. Load Model with Full Fine-Tuning Enabled

Using `full_finetuning=True` to train all parameters.

In [ ]:
from unsloth import FastLanguageModel

print("Loading model for Full Fine-Tuning...")
print(f"  Model: {MODEL_NAME}")
print(f"  This will train ALL 7.6B parameters!")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (will use BF16)
    load_in_4bit=False,  # Full fine-tuning uses full precision
    full_finetuning=True,  # ← KEY: Enable full fine-tuning
)

mem_used = torch.cuda.memory_allocated() / 1e9

print(f"\n✓ Model loaded for full fine-tuning!")
print(f"  GPU memory used: {mem_used:.2f} GB")
print(f"  Tokenizer vocab size: {len(tokenizer)}")

## 4. Verify All Parameters Are Trainable

In [ ]:
# Count parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"Parameter Count:")
print(f"  Trainable: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"  Total: {total:,}")

if trainable == total:
    print(f"\n✓ All parameters are trainable (Full Fine-Tuning mode)")
else:
    print(f"\n⚠ Warning: Only {100*trainable/total:.2f}% of parameters are trainable")

## 5. Load Training Dataset

In [ ]:
from datasets import load_dataset

print(f"Loading dataset from: {TRAIN_DATA_PATH}")

dataset = load_dataset("json", data_files=TRAIN_DATA_PATH, split="train")

print(f"\nDataset loaded:")
print(f"  Total examples: {len(dataset):,}")
print(f"  Columns: {dataset.column_names}")

print(f"\nSample entry:")
sample = dataset[0]
for msg in sample["messages"]:
    role = msg["role"]
    content = msg["content"][:80] + "..." if len(msg["content"]) > 80 else msg["content"]
    print(f"  [{role}]: {content}")

## 6. Format Dataset

In [ ]:
def formatting_prompts_func(examples):
    """Format examples using the tokenizer's chat template."""
    texts = []
    for messages in examples["messages"]:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )
        texts.append(text)
    return {"text": texts}

print("Applying chat template to dataset...")
formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    num_proc=4,
    desc="Formatting",
)

print(f"\nFormatted dataset columns: {formatted_dataset.column_names}")
print(f"\nSample (first 400 chars):")
print(formatted_dataset[0]["text"][:400])

## 7. Configure Training

In [ ]:
from trl import SFTTrainer, SFTConfig

total_steps = (len(formatted_dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)) * NUM_EPOCHS

print(f"Training configuration:")
print(f"  Total examples: {len(formatted_dataset):,}")
print(f"  Batch size: {BATCH_SIZE} x {GRADIENT_ACCUMULATION_STEPS} = {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  Estimated total steps: {total_steps:,}")

sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    optim="adamw_8bit",
    bf16=True,
    fp16=False,
    max_length=MAX_SEQ_LENGTH,
    packing=True,
    logging_steps=LOGGING_STEPS,
    logging_first_step=True,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    dataloader_num_workers=4,
    gradient_checkpointing=True,  # Essential for full fine-tuning
    seed=42,
    report_to="none",
)

print("\n✓ SFTConfig created!")

## 8. Create Trainer and Start Training

In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=formatted_dataset,
    args=sft_config,
)

print("✓ Trainer created!")
print(f"\nStarting Full Fine-Tuning...")
print("=" * 60)

In [ ]:
import time

start_time = time.time()

trainer_stats = trainer.train()

elapsed_time = time.time() - start_time
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)

print("\n" + "=" * 60)
print("Full Fine-Tuning Complete!")
print("=" * 60)
print(f"\nTraining time: {int(hours)}h {int(minutes)}m {int(seconds)}s")
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print(f"Total steps: {trainer_stats.global_step}")

## 9. Save the Full Model

In [ ]:
model_path = f"{OUTPUT_DIR}/final"

print(f"Saving full fine-tuned model to: {model_path}")

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Check saved files
import os
saved_files = os.listdir(model_path)
total_size = sum(os.path.getsize(os.path.join(model_path, f)) for f in saved_files if os.path.isfile(os.path.join(model_path, f)))

print(f"\nSaved files:")
for f in sorted(saved_files)[:10]:  # Show first 10 files
    path = os.path.join(model_path, f)
    if os.path.isfile(path):
        size = os.path.getsize(path)
        print(f"  {f}: {size / 1e9:.2f} GB" if size > 1e9 else f"  {f}: {size / 1e6:.2f} MB")

print(f"\nTotal model size: {total_size / 1e9:.2f} GB")
print(f"\n✓ Full model saved!")

## 10. Quick Evaluation

In [ ]:
FastLanguageModel.for_inference(model)

SYSTEM_PROMPT = """You are a news article classifier. Categorize into: World, Sports, Business, or Sci/Tech.
Respond with JSON: {"category": "<category>"}"""

test_articles = [
    ("The Federal Reserve announced a quarter-point interest rate cut.", "Business"),
    ("Scientists at CERN discovered a new subatomic particle.", "Sci/Tech"),
    ("The Lakers defeated the Celtics 112-108 in overtime.", "Sports"),
    ("The UN Security Council voted to impose new sanctions.", "World"),
]

print("Testing full fine-tuned model:")
print("=" * 60)

correct = 0
for article, expected in test_articles:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Classify: {article}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True,
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=50,
        temperature=0.0,
        do_sample=False,
    )
    
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    is_correct = expected.lower() in response.lower()
    if is_correct:
        correct += 1
    
    print(f"\nArticle: {article[:50]}...")
    print(f"Expected: {expected}")
    print(f"Response: {response.strip()}")
    print(f"Status: {'✓' if is_correct else '✗'}")

print(f"\n" + "=" * 60)
print(f"Quick test accuracy: {correct}/{len(test_articles)} ({100*correct/len(test_articles):.0f}%)")

## Conclusions

*To be filled after running the notebook*

### Training Results

| Metric | Value |
|--------|-------|
| Training Time | TBD |
| Final Loss | TBD |
| Total Steps | TBD |
| Model Size | TBD |
| GPU Memory | TBD |

### Performance Comparison

| Metric | HuggingFace Full | Unsloth Full |
|--------|------------------|---------------|
| Tokens/sec | ~900 | TBD |
| Training time | ~10h | TBD |